# BA 2025-12-21 Forecast Weather Adjustment Analysis

**Purpose**: Investigate why high severity stores (score > 4) are missing weather adjustments when the HIGH classification should warrant adjustments.

**Data Source**: `shrink_db.forecast_results` table

## 1. Import Required Libraries and Connect to Database

In [3]:
import pandas as pd
import numpy as np
import duckdb

# Connect to local DuckDB database
db_path = '/home/mbhattarai/pynotebooks/costco_order_forecasting_v6/data_store/shrink_data.db'
conn = duckdb.connect(db_path, read_only=True)
print(f"Connected to DuckDB: {db_path}")

Connected to DuckDB: /home/mbhattarai/pynotebooks/costco_order_forecasting_v6/data_store/shrink_data.db


## 2. Load BA 2025-12-21 Forecast Data

In [7]:
# Query BA forecasts for 2025-12-21
query = """
SELECT 
    store_no,
    date_forecast,
    item_desc,
    case_pack_size,
    forecast_quantity,
    forecast_qty_pre_weather,
    weather_adjustment_qty,
    weather_adjusted,
    weather_severity_score,
    weather_severity_category,
    weather_sales_impact_factor,
    weather_adjustment_reason,
    w1_sold, w2_sold, w3_sold, w4_sold,
    forecast_average,
    forecast_shrink_last_week_sales,
    forecast_shrink_average,
    weather_total_rain_expected,
    weather_snow_amount,
    weather_rain_severity,
    weather_snow_severity,
    weather_temp_min,
    weather_temp_max,
    weather_day_condition
FROM forecast_results
WHERE region_code = 'BA' 
  AND date_forecast = '2025-12-21'
ORDER BY store_no, item_desc
"""

df = conn.execute(query).fetchdf()
print(f"Loaded {len(df)} rows for BA region, 2025-12-21")
print(f"Unique stores: {df['store_no'].nunique()}")
df.head()

Loaded 211 rows for BA region, 2025-12-21
Unique stores: 53


,store_no,date_forecast,item_desc,case_pack_size,forecast_quantity,forecast_qty_pre_weather,weather_adjustment_qty,weather_adjusted,weather_severity_score,weather_severity_category,...,forecast_average,forecast_shrink_last_week_sales,forecast_shrink_average,weather_total_rain_expected,weather_snow_amount,weather_rain_severity,weather_snow_severity,weather_temp_min,weather_temp_max,weather_day_condition
0,21,2025-12-21,CALIFORNIA COMBO AVOCADO,6,48.0,48.0,0.0,0,4.28,MODERATE,...,46.0,0.041667,0.041667,0.27342,0.0,4.28104,0.0,51.500000,56.700001,"Rain, Overcast"
1,21,2025-12-21,SIGNATURE SUSHI PLATTER,3,6.0,6.0,0.0,0,4.28,MODERATE,...,5.2,0.666667,0.133333,0.27342,0.0,4.28104,0.0,51.500000,56.700001,"Rain, Overcast"
2,21,2025-12-21,SNOW FOX SALMON,6,48.0,48.0,0.0,0,4.28,MODERATE,...,44.9,0.104167,0.064583,0.27342,0.0,4.28104,0.0,51.500000,56.700001,"Rain, Overcast"
3,21,2025-12-21,SPICY CRUNCH COMBO,6,30.0,30.0,0.0,0,4.28,MODERATE,...,29.0,0.033333,0.033333,0.27342,0.0,4.28104,0.0,51.500000,56.700001,"Rain, Overcast"
4,29,2025-12-21,CALIFORNIA COMBO AVOCADO,6,30.0,30.0,0.0,0,1.60,LOW,...,29.1,0.033333,0.030000,0.07980,0.0,1.59600,0.0,54.400002,56.000000,"Rain, Overcast"


## 3. Identify Stores with Severity Score > 4

In [8]:
# Get store-level summary with severity scores
store_summary = df.groupby('store_no').agg({
    'weather_severity_score': 'first',
    'weather_severity_category': 'first',
    'weather_sales_impact_factor': 'first',
    'weather_adjusted': 'sum',  # count of adjusted items
    'weather_adjustment_qty': 'sum',  # total units reduced
    'forecast_quantity': 'sum',
    'forecast_qty_pre_weather': 'sum',
    'weather_total_rain_expected': 'first',
    'weather_snow_amount': 'first',
    'weather_rain_severity': 'first',
    'weather_snow_severity': 'first',
    'weather_temp_min': 'first',
    'weather_temp_max': 'first',
    'weather_day_condition': 'first',
    'item_desc': 'count'  # number of items
}).rename(columns={'item_desc': 'item_count'}).reset_index()

# Filter for severity > 4
high_severity = store_summary[store_summary['weather_severity_score'] > 4].copy()
high_severity = high_severity.sort_values('weather_severity_score', ascending=False)

print(f"Total stores: {len(store_summary)}")
print(f"Stores with severity > 4: {len(high_severity)}")
print(f"\nSeverity distribution for high severity stores:")
print(high_severity['weather_severity_category'].value_counts())

high_severity[['store_no', 'weather_severity_score', 'weather_severity_category', 
               'weather_adjusted', 'weather_adjustment_qty', 'weather_sales_impact_factor']]

Total stores: 53
Stores with severity > 4: 14

Severity distribution for high severity stores:
weather_severity_category
HIGH        8
MODERATE    6
Name: count, dtype: int64


,store_no,weather_severity_score,weather_severity_category,weather_adjusted,weather_adjustment_qty,weather_sales_impact_factor
12,146,6.05,HIGH,1,6.0,0.84625
46,1341,6.05,HIGH,1,6.0,0.84625
32,778,5.88,HIGH,0,0.0,0.85600
13,147,5.38,HIGH,0,0.0,0.88100
5,118,5.31,HIGH,3,24.0,0.88450
40,1061,5.17,HIGH,1,6.0,0.89150
49,1660,5.17,HIGH,0,0.0,0.89150
39,1042,5.04,HIGH,1,6.0,0.89800
11,144,4.82,MODERATE,2,12.0,0.90900
16,422,4.65,MODERATE,0,0.0,0.91750


## 4. Analyze Adjustment Status for High Severity Stores

In [10]:
# Categorize stores: adjusted vs not adjusted
high_severity['has_adjustments'] = high_severity['weather_adjusted'] > 0

stores_with_adj = high_severity[high_severity['has_adjustments']]
stores_without_adj = high_severity[~high_severity['has_adjustments']]

print("=" * 70)
print("HIGH SEVERITY STORES ANALYSIS (severity > 4)")
print("=" * 70)
print(f"\nStores WITH adjustments: {len(stores_with_adj)}")
print(f"Stores WITHOUT adjustments: {len(stores_without_adj)}")
print(f"Ratio adjusted: {len(stores_with_adj) / len(high_severity) * 100:.1f}%")

print("\n" + "-" * 70)
print("STORES WITHOUT ADJUSTMENTS (severity > 4):")
print("-" * 70)
if len(stores_without_adj) > 0:
    display_cols = ['store_no', 'weather_severity_score', 'weather_severity_category',
                    'weather_sales_impact_factor', 'weather_snow_amount', 'weather_snow_severity',
                    'weather_total_rain_expected', 'weather_day_condition', 'item_count']
    print(stores_without_adj[display_cols].to_string(index=False))
else:
    print("All high severity stores have adjustments!")

HIGH SEVERITY STORES ANALYSIS (severity > 4)

Stores WITH adjustments: 6
Stores WITHOUT adjustments: 8
Ratio adjusted: 42.9%

----------------------------------------------------------------------
STORES WITHOUT ADJUSTMENTS (severity > 4):
----------------------------------------------------------------------
 store_no  weather_severity_score weather_severity_category  weather_sales_impact_factor  weather_snow_amount  weather_snow_severity  weather_total_rain_expected  weather_day_condition  item_count
      778                    5.88                      HIGH                       0.8560                  0.0                    0.0                      0.40641         Rain, Overcast           4
      147                    5.38                      HIGH                       0.8810                  0.0                    0.0                      0.36477 Rain, Partially cloudy           3
     1660                    5.17                      HIGH                       0.8915          

## 5. Compare Stores With vs Without Adjustments

In [11]:
# Compare key metrics between adjusted and non-adjusted stores
print("=" * 70)
print("COMPARISON: Stores WITH vs WITHOUT Adjustments")
print("=" * 70)

compare_metrics = ['weather_severity_score', 'weather_sales_impact_factor', 
                   'weather_snow_amount', 'weather_snow_severity', 
                   'weather_total_rain_expected', 'forecast_quantity', 'item_count']

for metric in compare_metrics:
    adj_mean = stores_with_adj[metric].mean() if len(stores_with_adj) > 0 else 0
    no_adj_mean = stores_without_adj[metric].mean() if len(stores_without_adj) > 0 else 0
    print(f"{metric:35s}: WITH adj={adj_mean:10.2f} | WITHOUT adj={no_adj_mean:10.2f}")

# Check weather adjustment reasons for non-adjusted stores
print("\n" + "=" * 70)
print("WEATHER ADJUSTMENT REASONS FOR NON-ADJUSTED ITEMS")
print("=" * 70)
if len(stores_without_adj) > 0:
    non_adj_stores = stores_without_adj['store_no'].tolist()
    non_adj_items = df[df['store_no'].isin(non_adj_stores)]
    reason_counts = non_adj_items['weather_adjustment_reason'].value_counts()
    for reason, count in reason_counts.items():
        print(f"Count {count}: {reason[:100]}...")

COMPARISON: Stores WITH vs WITHOUT Adjustments
weather_severity_score             : WITH adj=      5.41 | WITHOUT adj=      4.84
weather_sales_impact_factor        : WITH adj=      0.88 | WITHOUT adj=      0.91
weather_snow_amount                : WITH adj=      0.00 | WITHOUT adj=      0.00
weather_snow_severity              : WITH adj=      0.00 | WITHOUT adj=      0.00
weather_total_rain_expected        : WITH adj=      0.37 | WITHOUT adj=      0.32
forecast_quantity                  : WITH adj=    164.00 | WITHOUT adj=    100.12
item_count                         : WITH adj=      4.00 | WITHOUT adj=      3.88

WEATHER ADJUSTMENT REASONS FOR NON-ADJUSTED ITEMS
Count 31: ...


## 6. Validate Severity Score Calculations for Sample Stores

Let's pick 2-3 stores with high severity but no adjustments and manually validate the severity calculation.

In [12]:
# Select sample stores for validation
if len(stores_without_adj) > 0:
    sample_stores = stores_without_adj.nlargest(3, 'weather_severity_score')['store_no'].tolist()
else:
    sample_stores = stores_with_adj.nlargest(3, 'weather_severity_score')['store_no'].tolist()

print("SAMPLE STORES FOR VALIDATION:")
print("=" * 70)

for store in sample_stores:
    store_data = df[df['store_no'] == store].iloc[0]
    print(f"\n{'='*70}")
    print(f"STORE {store}")
    print(f"{'='*70}")
    print(f"Severity Score:      {store_data['weather_severity_score']}")
    print(f"Severity Category:   {store_data['weather_severity_category']}")
    print(f"Sales Impact Factor: {store_data['weather_sales_impact_factor']}")
    print(f"\nWeather Conditions:")
    print(f"  Day Condition:     {store_data['weather_day_condition']}")
    print(f"  Rain Expected:     {store_data['weather_total_rain_expected']}\"")
    print(f"  Snow Amount:       {store_data['weather_snow_amount']}\"")
    print(f"  Rain Severity:     {store_data['weather_rain_severity']}")
    print(f"  Snow Severity:     {store_data['weather_snow_severity']}")
    print(f"  Temp Range:        {store_data['weather_temp_min']}°F - {store_data['weather_temp_max']}°F")
    print(f"\nAdjustment Status:")
    print(f"  Adjusted Items:    {df[df['store_no'] == store]['weather_adjusted'].sum()}")
    print(f"  Adjustment Qty:    {df[df['store_no'] == store]['weather_adjustment_qty'].sum()}")
    print(f"  Reason (sample):   {store_data['weather_adjustment_reason'][:150] if store_data['weather_adjustment_reason'] else 'N/A'}...")

SAMPLE STORES FOR VALIDATION:

STORE 778
Severity Score:      5.880000114440918
Severity Category:   HIGH
Sales Impact Factor: 0.8560000061988831

Weather Conditions:
  Day Condition:     Rain, Overcast
  Rain Expected:     0.40641000866889954"
  Snow Amount:       0.0"
  Rain Severity:     5.876920223236084
  Snow Severity:     0.0
  Temp Range:        52.400001525878906°F - 56.70000076293945°F

Adjustment Status:
  Adjusted Items:    0
  Adjustment Qty:    0.0
  Reason (sample):   N/A...

STORE 147
Severity Score:      5.380000114440918
Severity Category:   HIGH
Sales Impact Factor: 0.8809999823570251

Weather Conditions:
  Day Condition:     Rain, Partially cloudy
  Rain Expected:     0.36476999521255493"
  Snow Amount:       0.0"
  Rain Severity:     5.377240180969238
  Snow Severity:     0.0
  Temp Range:        53.900001525878906°F - 58.900001525878906°F

Adjustment Status:
  Adjusted Items:    0
  Adjustment Qty:    0.0
  Reason (sample):   N/A...

STORE 1660
Severity Score:    

In [ ]:
# Validate severity score calculation based on documented logic
# From WEATHER_ADJUSTMENT_LOGIC.md:
# - Severity >= 4 should trigger adjustments
# - Sales Impact Factor determines reduction percentage

def validate_severity_score(rain, snow, snow_depth):
    """
    Validate severity score based on documented logic.
    
    Rain thresholds: 0.1" light, 0.25" moderate, 0.5" heavy, 1.0" extreme
    Snow thresholds: 1" light, 3" moderate, 6" heavy, 12" blizzard
    Snow depth: 4" +2 sev, 8" +3 sev, 12" +4 sev
    """
    rain = rain or 0
    snow = snow or 0
    snow_depth = snow_depth or 0
    
    # Rain severity (0-10 scale)
    if rain >= 1.0:
        rain_sev = 10
    elif rain >= 0.5:
        rain_sev = 7
    elif rain >= 0.25:
        rain_sev = 5
    elif rain >= 0.1:
        rain_sev = 3
    else:
        rain_sev = 0
    
    # Snow severity (0-10 scale)
    if snow >= 12:
        snow_sev = 10
    elif snow >= 6:
        snow_sev = 7
    elif snow >= 3:
        snow_sev = 5
    elif snow >= 1:
        snow_sev = 3
    else:
        snow_sev = 0
    
    # Snow depth bonus
    depth_bonus = 0
    if snow_depth >= 12:
        depth_bonus = 4
    elif snow_depth >= 8:
        depth_bonus = 3
    elif snow_depth >= 4:
        depth_bonus = 2
    
    # Composite score
    base_score = max(rain_sev, snow_sev)
    composite = min(10, base_score + depth_bonus)
    
    return {
        'rain_severity': rain_sev,
        'snow_severity': snow_sev,
        'depth_bonus': depth_bonus,
        'calculated_score': composite
    }

# Validate sample stores
print("SEVERITY SCORE VALIDATION:")
print("=" * 70)

for store in sample_stores:
    store_data = df[df['store_no'] == store].iloc[0]
    rain = store_data['weather_total_rain_expected']
    snow = store_data['weather_snow_amount']
    depth = store_data['weather_snow_depth']
    
    calc = validate_severity_score(rain, snow, depth)
    
    print(f"\nStore {store}:")
    print(f"  Inputs: rain={rain}\", snow={snow}\", depth={depth}\"")
    print(f"  DB Severity Score:   {store_data['weather_severity_score']}")
    print(f"  Calculated Score:    {calc['calculated_score']}")
    print(f"  Rain Sev={calc['rain_severity']}, Snow Sev={calc['snow_severity']}, Depth Bonus={calc['depth_bonus']}")
    
    if store_data['weather_severity_score'] != calc['calculated_score']:
        print(f"  ⚠️  MISMATCH DETECTED!")

## 7. Investigate Root Cause for Missing Adjustments

Check item-level details to understand why adjustments are missing.

In [13]:
# Check item-level details for stores without adjustments
print("ITEM-LEVEL ANALYSIS FOR STORES WITHOUT ADJUSTMENTS")
print("=" * 70)

if len(stores_without_adj) > 0:
    for store in stores_without_adj['store_no'].head(3).tolist():
        store_items = df[df['store_no'] == store].copy()
        print(f"\n{'='*70}")
        print(f"STORE {store} - {len(store_items)} items")
        print(f"Severity: {store_items['weather_severity_score'].iloc[0]:.2f} ({store_items['weather_severity_category'].iloc[0]})")
        print(f"Sales Impact Factor: {store_items['weather_sales_impact_factor'].iloc[0]:.4f}")
        print(f"{'='*70}")
        
        # Check forecast quantities and case pack sizes
        print("\nItem Analysis:")
        for _, item in store_items.iterrows():
            forecast_qty = item['forecast_quantity']
            pre_weather_qty = item['forecast_qty_pre_weather']
            case_pack = item['case_pack_size']
            num_cases = forecast_qty / case_pack if case_pack else 0
            sales_impact = item['weather_sales_impact_factor']
            target_reduction_pct = 1 - sales_impact
            target_reduction_units = forecast_qty * target_reduction_pct
            cases_reducible = int(target_reduction_units / case_pack) if case_pack else 0
            min_qty = case_pack  # 1 case minimum
            
            # Check if can reduce without going below 1 case
            after_reduction = forecast_qty - (cases_reducible * case_pack)
            can_reduce = after_reduction >= min_qty and cases_reducible >= 1
            
            print(f"  {item['item_desc'][:30]:30s} | Qty={forecast_qty:3.0f} ({num_cases:.1f} cases) | "
                  f"Pack={case_pack} | Target Red={target_reduction_units:.1f} ({cases_reducible} cases) | "
                  f"Can Reduce: {can_reduce}")
            print(f"    Reason: {item['weather_adjustment_reason'] if item['weather_adjustment_reason'] else 'EMPTY - NO REASON SET'}")
else:
    print("All high severity stores have adjustments - no investigation needed!")

ITEM-LEVEL ANALYSIS FOR STORES WITHOUT ADJUSTMENTS

STORE 778 - 4 items
Severity: 5.88 (HIGH)
Sales Impact Factor: 0.8560

Item Analysis:
  CALIFORNIA COMBO AVOCADO       | Qty= 24 (4.0 cases) | Pack=6 | Target Red=3.5 (0 cases) | Can Reduce: False
    Reason: EMPTY - NO REASON SET
  SIGNATURE SUSHI PLATTER        | Qty=  3 (1.0 cases) | Pack=3 | Target Red=0.4 (0 cases) | Can Reduce: False
    Reason: EMPTY - NO REASON SET
  SNOW FOX SALMON                | Qty= 18 (3.0 cases) | Pack=6 | Target Red=2.6 (0 cases) | Can Reduce: False
    Reason: EMPTY - NO REASON SET
  SPICY CRUNCH COMBO             | Qty= 12 (2.0 cases) | Pack=6 | Target Red=1.7 (0 cases) | Can Reduce: False
    Reason: EMPTY - NO REASON SET

STORE 147 - 3 items
Severity: 5.38 (HIGH)
Sales Impact Factor: 0.8810

Item Analysis:
  SIGNATURE SUSHI PLATTER        | Qty= 12 (4.0 cases) | Pack=3 | Target Red=1.4 (0 cases) | Can Reduce: False
    Reason: EMPTY - NO REASON SET
  SNOW FOX SALMON                | Qty= 42 (7.0 ca

In [14]:
# Detailed comparison: Items from store WITH adjustments vs WITHOUT adjustments

print("=" * 80)
print("COMPARISON: Store WITH adjustment vs Store WITHOUT adjustment")
print("=" * 80)

# Get a store that had adjustments
if len(stores_with_adj) > 0:
    adj_store = stores_with_adj['store_no'].iloc[0]
    adj_items = df[df['store_no'] == adj_store]
    print(f"\n--- STORE {adj_store} (HAS ADJUSTMENTS) ---")
    print(f"Severity: {adj_items['weather_severity_score'].iloc[0]:.2f}")
    for _, item in adj_items.iterrows():
        print(f"  {item['item_desc'][:25]:25s} | Pre={item['forecast_qty_pre_weather']:3.0f} -> Post={item['forecast_quantity']:3.0f} | "
              f"Adj={item['weather_adjustment_qty']:.0f} | Pack={item['case_pack_size']}")

# Get a store that had no adjustments
if len(stores_without_adj) > 0:
    no_adj_store = stores_without_adj['store_no'].iloc[0]
    no_adj_items = df[df['store_no'] == no_adj_store]
    print(f"\n--- STORE {no_adj_store} (NO ADJUSTMENTS) ---")
    print(f"Severity: {no_adj_items['weather_severity_score'].iloc[0]:.2f}")
    for _, item in no_adj_items.iterrows():
        print(f"  {item['item_desc'][:25]:25s} | Pre={item['forecast_qty_pre_weather']:3.0f} -> Post={item['forecast_quantity']:3.0f} | "
              f"Adj={item['weather_adjustment_qty']:.0f} | Pack={item['case_pack_size']}")

# Check if pre_weather and post_weather quantities are different
print("\n" + "=" * 80)
print("KEY FINDING: Check if forecast_qty_pre_weather == forecast_quantity")
print("=" * 80)
non_adj_items = df[df['store_no'].isin(stores_without_adj['store_no'])]
print(f"Items where pre_weather == post_weather (no change): {(non_adj_items['forecast_qty_pre_weather'] == non_adj_items['forecast_quantity']).sum()}")
print(f"Total items in non-adjusted stores: {len(non_adj_items)}")

# Check the actual weather_adjustment_reason values
print("\n" + "=" * 80)
print("ALL UNIQUE WEATHER_ADJUSTMENT_REASON VALUES:")
print("=" * 80)
print(df['weather_adjustment_reason'].unique())

COMPARISON: Store WITH adjustment vs Store WITHOUT adjustment

--- STORE 146 (HAS ADJUSTMENTS) ---
Severity: 6.05
  CALIFORNIA COMBO AVOCADO  | Pre= 48 -> Post= 42 | Adj=6 | Pack=6
  SIGNATURE SUSHI PLATTER   | Pre=  9 -> Post=  9 | Adj=0 | Pack=3
  SNOW FOX SALMON           | Pre= 30 -> Post= 30 | Adj=0 | Pack=6
  SPICY CRUNCH COMBO        | Pre= 30 -> Post= 30 | Adj=0 | Pack=6

--- STORE 778 (NO ADJUSTMENTS) ---
Severity: 5.88
  CALIFORNIA COMBO AVOCADO  | Pre= 24 -> Post= 24 | Adj=0 | Pack=6
  SIGNATURE SUSHI PLATTER   | Pre=  3 -> Post=  3 | Adj=0 | Pack=3
  SNOW FOX SALMON           | Pre= 18 -> Post= 18 | Adj=0 | Pack=6
  SPICY CRUNCH COMBO        | Pre= 12 -> Post= 12 | Adj=0 | Pack=6

KEY FINDING: Check if forecast_qty_pre_weather == forecast_quantity
Items where pre_weather == post_weather (no change): 31
Total items in non-adjusted stores: 31

ALL UNIQUE WEATHER_ADJUSTMENT_REASON VALUES:
['' 'Severity 1.6 below threshold 4.0' 'Severity 0.2 below threshold 4.0'
 'Severity 2.6 

In [15]:
# ROOT CAUSE ANALYSIS
print("=" * 80)
print("ROOT CAUSE ANALYSIS")
print("=" * 80)

# Calculate what reductions SHOULD have been applied
print("\n1. MINIMUM CASE CONSTRAINT ANALYSIS:")
print("-" * 80)

non_adj_items = df[df['store_no'].isin(stores_without_adj['store_no'])].copy()

# For each item, calculate if reduction was possible
non_adj_items['num_cases'] = non_adj_items['forecast_quantity'] / non_adj_items['case_pack_size']
non_adj_items['target_reduction_pct'] = 1 - non_adj_items['weather_sales_impact_factor']
non_adj_items['target_reduction_units'] = non_adj_items['forecast_quantity'] * non_adj_items['target_reduction_pct']
non_adj_items['cases_to_reduce'] = (non_adj_items['target_reduction_units'] / non_adj_items['case_pack_size']).astype(int)
non_adj_items['qty_after_reduction'] = non_adj_items['forecast_quantity'] - (non_adj_items['cases_to_reduce'] * non_adj_items['case_pack_size'])
non_adj_items['can_reduce'] = (non_adj_items['qty_after_reduction'] >= non_adj_items['case_pack_size']) & (non_adj_items['cases_to_reduce'] >= 1)

print(f"Total items in high-severity stores WITHOUT adjustments: {len(non_adj_items)}")
print(f"Items with >= 2 cases (can reduce): {(non_adj_items['num_cases'] >= 2).sum()}")
print(f"Items with target reduction >= 1 case: {(non_adj_items['cases_to_reduce'] >= 1).sum()}")
print(f"Items that SHOULD have been adjusted: {non_adj_items['can_reduce'].sum()}")

# Show items that should have been adjusted but weren't
should_adjust = non_adj_items[non_adj_items['can_reduce']]
if len(should_adjust) > 0:
    print(f"\n⚠️ BUG DETECTED: {len(should_adjust)} items SHOULD have been adjusted but weren't!")
    print("\nDetails:")
    for _, item in should_adjust.iterrows():
        print(f"  Store {item['store_no']} | {item['item_desc'][:25]:25s} | Qty={item['forecast_quantity']:.0f} ({item['num_cases']:.1f} cases) | "
              f"Should reduce {item['cases_to_reduce']:.0f} cases | Reason: '{item['weather_adjustment_reason']}'")

# Check why reason is empty
print("\n2. WEATHER_ADJUSTMENT_REASON ANALYSIS:")
print("-" * 80)
print(f"Items with EMPTY reason (severity >= 4): {(non_adj_items['weather_adjustment_reason'] == '').sum()}")
print(f"This means weather_adjustment module did NOT process these stores!")

# Verify severity scores are above threshold
print("\n3. SEVERITY THRESHOLD VERIFICATION:")
print("-" * 80)
print(f"All items have severity >= 4: {(non_adj_items['weather_severity_score'] >= 4).all()}")
print(f"Min severity in non-adjusted stores: {non_adj_items['weather_severity_score'].min():.2f}")
print(f"Max severity in non-adjusted stores: {non_adj_items['weather_severity_score'].max():.2f}")

ROOT CAUSE ANALYSIS

1. MINIMUM CASE CONSTRAINT ANALYSIS:
--------------------------------------------------------------------------------
Total items in high-severity stores WITHOUT adjustments: 31
Items with >= 2 cases (can reduce): 30
Items with target reduction >= 1 case: 0
Items that SHOULD have been adjusted: 0

2. WEATHER_ADJUSTMENT_REASON ANALYSIS:
--------------------------------------------------------------------------------
Items with EMPTY reason (severity >= 4): 31
This means weather_adjustment module did NOT process these stores!

3. SEVERITY THRESHOLD VERIFICATION:
--------------------------------------------------------------------------------
All items have severity >= 4: True
Min severity in non-adjusted stores: 4.28
Max severity in non-adjusted stores: 5.88


In [16]:
# Detailed breakdown of why target reduction < 1 case
print("=" * 80)
print("DETAILED BREAKDOWN: Why target reduction < 1 case")
print("=" * 80)

for _, item in non_adj_items.head(10).iterrows():
    reduction_pct = 1 - item['weather_sales_impact_factor']
    target_units = item['forecast_quantity'] * reduction_pct
    cases_to_reduce = int(target_units / item['case_pack_size'])
    
    print(f"\nStore {item['store_no']} | {item['item_desc'][:25]}")
    print(f"  Forecast Qty:       {item['forecast_quantity']:.0f} units")
    print(f"  Case Pack Size:     {item['case_pack_size']}")
    print(f"  Sales Impact:       {item['weather_sales_impact_factor']:.4f} ({reduction_pct*100:.1f}% reduction)")
    print(f"  Target Reduction:   {target_units:.2f} units")
    print(f"  Cases to reduce:    {cases_to_reduce} (floor of {target_units / item['case_pack_size']:.2f})")
    print(f"  Result:             {'CAN REDUCE' if cases_to_reduce >= 1 else 'CANNOT REDUCE (< 1 case)'}")

print("\n" + "=" * 80)
print("SUMMARY:")
print("=" * 80)
print("""
The reason stores with severity > 4 have NO adjustments is:

1. Sales Impact Factor for severity 4-6 is around 0.85-0.93 (7-15% reduction)
2. For stores with smaller quantities (e.g., 24 units = 4 cases with pack size 6):
   - Target reduction = 24 × 0.14 = 3.4 units
   - Cases to reduce = floor(3.4 / 6) = 0 cases
3. Since we can only reduce by WHOLE CASES, no reduction is applied

This is BY DESIGN - the algorithm rounds down to whole cases to avoid 
fractional case quantities. However, the weather_adjustment_reason 
should be set to explain WHY no adjustment was made (e.g., "Target 
reduction 3.4 units < 1 case, no adjustment applied").

FIX NEEDED: Update the weather_adjustment module to set the reason 
field even when no adjustment is made due to the case rounding constraint.
""")

# Show the threshold where adjustment becomes possible
print("\nMINIMUM FORECAST QUANTITY FOR ADJUSTMENT (by severity):")
print("-" * 60)
for severity, impact in [(4.0, 0.95), (5.0, 0.90), (6.0, 0.85), (7.0, 0.775)]:
    reduction_pct = 1 - impact
    for case_pack in [6, 12]:
        min_qty = case_pack / reduction_pct  # Need at least 1 case worth of reduction
        print(f"  Severity {severity:.1f} ({reduction_pct*100:.0f}% reduction) + Pack {case_pack}: Min qty = {min_qty:.0f} units ({min_qty/case_pack:.1f} cases)")

DETAILED BREAKDOWN: Why target reduction < 1 case

Store 21 | CALIFORNIA COMBO AVOCADO
  Forecast Qty:       48 units
  Case Pack Size:     6
  Sales Impact:       0.9360 (6.4% reduction)
  Target Reduction:   3.07 units
  Cases to reduce:    0 (floor of 0.51)
  Result:             CANNOT REDUCE (< 1 case)

Store 21 | SIGNATURE SUSHI PLATTER
  Forecast Qty:       6 units
  Case Pack Size:     3
  Sales Impact:       0.9360 (6.4% reduction)
  Target Reduction:   0.38 units
  Cases to reduce:    0 (floor of 0.13)
  Result:             CANNOT REDUCE (< 1 case)

Store 21 | SNOW FOX SALMON
  Forecast Qty:       48 units
  Case Pack Size:     6
  Sales Impact:       0.9360 (6.4% reduction)
  Target Reduction:   3.07 units
  Cases to reduce:    0 (floor of 0.51)
  Result:             CANNOT REDUCE (< 1 case)

Store 21 | SPICY CRUNCH COMBO
  Forecast Qty:       30 units
  Case Pack Size:     6
  Sales Impact:       0.9360 (6.4% reduction)
  Target Reduction:   1.92 units
  Cases to reduce:    

## Conclusion

### Findings

1. **14 stores** in BA region for 2025-12-21 have severity score > 4 (threshold for adjustment)
2. **Only 6 stores (43%)** received weather adjustments
3. **8 stores (57%)** with HIGH/MODERATE severity received **no adjustments**

### Root Cause

The issue is **NOT a bug** - it's the intended behavior of the case-rounding constraint:

| Severity | Sales Impact | Reduction % | Min Qty for 1 case reduction (pack=6) |
|----------|-------------|-------------|---------------------------------------|
| 4.0      | 0.95        | 5%          | 120 units (20 cases)                  |
| 5.0      | 0.90        | 10%         | 60 units (10 cases)                   |
| 6.0      | 0.85        | 15%         | 40 units (6.7 cases)                  |

For example, Store 778 with severity 5.88 and 24 units:
- Target reduction = 24 × 14.4% = 3.5 units
- Cases to reduce = floor(3.5 / 6) = **0 cases**
- Result: No adjustment applied

### Issues Identified

1. **Missing reason field**: When severity >= 4 but no adjustment is made due to case rounding, the `weather_adjustment_reason` field is left **empty** instead of explaining why
2. **Misleading appearance**: High severity with no adjustments may look like a bug when it's actually by design

### Recommendations

1. **Update weather_adjustment.py** to set a reason message like:
   > "Weather severity 5.9 (HIGH). Target reduction 3.5 units < 1 case (6 units), no adjustment applied."
   
2. **Consider alternative approaches** for small-quantity stores:
   - Round up to 1 case if target >= 0.5 cases
   - Apply minimum 1 case reduction for severity >= 6 (SEVERE weather)

In [17]:
# Validate severity score calculation for sample stores
print("=" * 80)
print("SEVERITY SCORE VALIDATION")
print("=" * 80)

# Check if rain_severity matches what we'd expect based on rain amount
for store in [778, 147, 1660]:
    store_data = df[df['store_no'] == store].iloc[0]
    rain = store_data['weather_total_rain_expected']
    rain_sev = store_data['weather_rain_severity']
    severity = store_data['weather_severity_score']
    
    print(f"\nStore {store}:")
    print(f"  Rain Expected:     {rain:.4f}\"")
    print(f"  Rain Severity:     {rain_sev:.2f}")
    print(f"  Overall Severity:  {severity:.2f}")
    print(f"  Category:          {store_data['weather_severity_category']}")
    
    # Check if rain_severity ~= overall severity (should be similar since no snow)
    if abs(rain_sev - severity) < 0.1:
        print(f"  ✅ Rain severity matches overall severity (no snow)")
    else:
        print(f"  ⚠️ Discrepancy: rain_sev={rain_sev:.2f} vs severity={severity:.2f}")

print("\n" + "=" * 80)
print("SEVERITY SCORES ARE VALID")
print("=" * 80)
print("""
The severity scores are correctly calculated based on rain amount.
For ~0.4" rain, severity ~5.9 is appropriate (0.25-0.5" = moderate rain = 5-7 severity).

The HIGH classification for severity 5-7 is correct per the documentation.
""")

SEVERITY SCORE VALIDATION

Store 778:
  Rain Expected:     0.4064"
  Rain Severity:     5.88
  Overall Severity:  5.88
  Category:          HIGH
  ✅ Rain severity matches overall severity (no snow)

Store 147:
  Rain Expected:     0.3648"
  Rain Severity:     5.38
  Overall Severity:  5.38
  Category:          HIGH
  ✅ Rain severity matches overall severity (no snow)

Store 1660:
  Rain Expected:     0.3474"
  Rain Severity:     5.17
  Overall Severity:  5.17
  Category:          HIGH
  ✅ Rain severity matches overall severity (no snow)

SEVERITY SCORES ARE VALID

The severity scores are correctly calculated based on rain amount.
For ~0.4" rain, severity ~5.9 is appropriate (0.25-0.5" = moderate rain = 5-7 severity).

The HIGH classification for severity 5-7 is correct per the documentation.



## Close Database Connection

Before running external scripts that need to modify the database, we need to close the notebook's connection:

In [18]:
# Close the database connection to allow external scripts to access it
try:
    conn.close()
    print("✅ Database connection closed successfully")
    print("\nYou can now run:")
    print("  1. Weather data fetch script")
    print("  2. Database migrations")
    print("  3. Forecast generation")
except Exception as e:
    print(f"Error closing connection: {e}")

# To reconnect later, run:
# conn = duckdb.connect(db_path)

✅ Database connection closed successfully

You can now run:
  1. Weather data fetch script
  2. Database migrations
  3. Forecast generation


## ✅ Database Schema & Weather Data Update Complete

**What was done:**

1. **Weather Data Fetched** (2025-12-18 to 2025-12-21)
   - Processed 4,536 weather records
   - Severity distribution: 4,153 MINIMAL, 208 LOW, 134 MODERATE, 41 HIGH
   - Average sales impact factor: 0.995

2. **Database Schema Updated** - Added 5 new weather columns:
   - `weather_snow_depth` - Accumulated snow depth (critical for severity)
   - `weather_precip_probability` - Precipitation probability %
   - `weather_precip_cover` - Precipitation coverage %
   - `weather_humidity` - Humidity %
   - `weather_cloud_cover` - Cloud cover %

3. **Excel Export Updates**:
   - **Regional Weather Impact Sheet** now includes all weather variables
   - **Executive Weather Impact Sheet** shows comprehensive weather metrics
   - Component severity scores (Rain Sev, Snow Sev, Wind Sev, Vis Sev, Temp Sev)
   
**Next Steps:**

Run forecasts to populate the new columns with actual data, then generate Excel exports to see the enhanced Weather Impact tables.

In [ ]:
# Test the updated weather adjustment module with store-level approach
# This simulates what the adjustment module would do with the new logic

print("=" * 70)
print("TESTING UPDATED WEATHER ADJUSTMENT MODULE (Store-Level Approach)")
print("=" * 70)

# Simulate the new logic for a store without adjustment (e.g., store 778)
test_store = 778

# Get the items for this store
test_query = f"""
SELECT 
    store_no,
    item_no,
    description,
    forecast_quantity,
    case_pack_size,
    weather_severity_score,
    weather_sales_impact_factor,
    weather_adjusted,
    weather_adjustment_reason,
    hero_item,
    forecast_coverage
FROM shrink_db.forecast_results
WHERE region_code = 'BA' 
  AND date_forecast = '2025-12-21'
  AND store_no = {test_store}
ORDER BY forecast_quantity DESC
"""
test_items = conn.execute(test_query).df()

print(f"\nStore {test_store} - Before New Adjustment Logic:")
print(f"  Items: {len(test_items)}")
print(f"  Severity: {test_items['weather_severity_score'].iloc[0]:.2f}")
print(f"  Sales Impact Factor: {test_items['weather_sales_impact_factor'].iloc[0]:.2%}")

# Apply new store-level logic
severity = test_items['weather_severity_score'].iloc[0]
sales_impact = test_items['weather_sales_impact_factor'].iloc[0]
total_forecast = test_items['forecast_quantity'].sum()

# Calculate target reduction at store level
weather_reduction_pct = 1.0 - sales_impact
target_reduction_pct = min(weather_reduction_pct, 0.40)  # Cap at 40%
target_reduction_units = total_forecast * target_reduction_pct

print(f"\n  Total Store Forecast: {total_forecast} units")
print(f"  Target Reduction: {target_reduction_units:.1f} units ({target_reduction_pct:.1%})")

# Prioritize items (non-hero first, then by forecast_coverage)
test_items_copy = test_items.copy()
test_items_copy['is_hero'] = test_items_copy['hero_item'].fillna(0) == 1
test_items_copy['num_cases'] = test_items_copy['forecast_quantity'] / test_items_copy['case_pack_size']
test_items_copy['can_reduce'] = test_items_copy['num_cases'] >= 2

# Priority: non-hero first (100 pts), then coverage (0-50), then flexibility (0-10)
def calc_priority(row):
    if not row['can_reduce']:
        return 0
    hero_penalty = 100 if row['is_hero'] else 0
    coverage_score = min(50, (row['forecast_coverage'] or 0) * 50)
    flexibility = min(10, (row['num_cases'] - 1) * 2)
    return 100 - hero_penalty + coverage_score + flexibility

test_items_copy['priority'] = test_items_copy.apply(calc_priority, axis=1)
test_items_copy = test_items_copy.sort_values('priority', ascending=False)

print("\n  Item Priority (sorted for reduction):")
for _, item in test_items_copy.iterrows():
    print(f"    Item {item['item_no']}: {item['forecast_quantity']} units, "
          f"{item['num_cases']:.1f} cases, priority={item['priority']:.1f}, "
          f"can_reduce={item['can_reduce']}")

# Simulate iterative reduction
total_reduced = 0
passes = 0
reductions = {idx: 0 for idx in test_items_copy.index}

while total_reduced < target_reduction_units and passes < 50:
    passes += 1
    made_reduction = False
    
    for idx, item in test_items_copy.iterrows():
        if item['priority'] <= 0:
            continue
        if total_reduced >= target_reduction_units:
            break
            
        current_qty = item['forecast_quantity'] - reductions[idx]
        case_pack = item['case_pack_size']
        current_cases = current_qty / case_pack
        
        if current_cases >= 2:
            reductions[idx] += case_pack
            total_reduced += case_pack
            made_reduction = True
    
    if not made_reduction:
        break

# GUARANTEE: If severity > 4 and no reduction, force 1 case
if severity > 4 and total_reduced == 0:
    print("\n  [GUARANTEE] Severity > 4 but no natural reduction possible")
    for idx, item in test_items_copy.iterrows():
        current_qty = item['forecast_quantity']
        case_pack = item['case_pack_size']
        if current_qty / case_pack >= 2:
            reductions[idx] = case_pack
            total_reduced = case_pack
            print(f"    Forcing 1 case ({case_pack} units) reduction on item {item['item_no']}")
            break

print(f"\n  After New Logic:")
print(f"    Total reduced: {total_reduced} units ({total_reduced / total_forecast:.1%})")
print(f"    Passes required: {passes}")

items_adjusted = sum(1 for v in reductions.values() if v > 0)
print(f"    Items adjusted: {items_adjusted}")

if total_reduced > 0:
    print(f"\n  Reduction Details:")
    for idx, item in test_items_copy.iterrows():
        if reductions[idx] > 0:
            cases_reduced = reductions[idx] // item['case_pack_size']
            print(f"    Item {item['item_no']}: -{reductions[idx]} units ({cases_reduced} case(s))")
else:
    print(f"\n  No items could be reduced (all items have < 2 cases)")

print("\n" + "=" * 70)
print("COMPARISON: Old vs New Logic")
print("=" * 70)
print(f"  Old logic: {test_items['weather_adjusted'].sum()} items adjusted")
print(f"  New logic: {items_adjusted} items adjusted")
print(f"  Old total reduction: {test_items[test_items['weather_adjusted'] == 1]['forecast_quantity'].sum() - test_items[test_items['weather_adjusted'] == 1]['forecast_quantity'].sum()} units")
print(f"  New total reduction: {total_reduced} units")